# Profiling tiling code for pgSTAC + COG

A pgSTAC database stores metadata about CMIP6 COGs on S3. The libraries used were pgstac for reading STAC metadata and rio_tiler's rasterio for reading COGs on S3.

In this notebook we load results from https://github.com/developmentseed/tile-benchmarking/blob/main/profiling/profile.ipynb to demonstrate:

1. The importance of GDAL variables in performance.
2. Variation across tiles is not significant.
3. Tiling with pgSTAC + COGs is fast when compared with titiler-xarray tiling of Zarr stores.

In [7]:
import pandas as pd
git_url_path = "https://raw.githubusercontent.com/developmentseed/tile-benchmarking/feat/fake-data/profiling/results"
pd.read_csv(f"{git_url_path}/pgstac_cog_gdal_results.csv")

,gdal_vars_set?,tile times,mean total time
0,with_gdal_vars,"[63.41, 54.53, 54.46]",57.466667
1,without_gdal_vars,"[14687.78, 30817.34, 17722.72]",21075.946667


We don't need many iterations since the variation is so great. You can see that setting GDAL environment variables makes things at least 100x faster.

These GDAL variables are documented here https://developmentseed.org/titiler/advanced/performance_tuning/, but that advice is copied into comments below for ease of reference.

By setting the GDAL environment variables we limit the number of total requests to S3.

Specifically, these environment variables ensure that:

* All of the metadata may be read in 1 request. This is not necessarily true, but more likely since we increase the initial number of GDAL ingested bytes.
* There is no superfluous LIST request to account for sidecar files, which don't exist for COGs.
* Consecutive range requests are merged into 1 request.
* Multiple range requests use the same TCP connection.

# Time to create different tiles

The difference between different tiles is very small.

In [8]:
pd.read_csv(f"{git_url_path}/pgstac_cog_tile_results.csv")

xyz tile                                         tile times  \
0   0           0  [85.67, 50.47, 50.97, 50.64, 49.72, 49.43, 49....   
    0           1  [57.45, 47.51, 45.7, 44.94, 46.92, 45.76, 43.2...   
1   1           2  [79.94, 44.41, 45.5, 44.44, 41.87, 40.93, 41.4...   
2   3           3  [72.54, 41.9, 41.29, 41.86, 42.19, 44.12, 42.8...   
4   6           4  [50.07, 38.15, 38.08, 39.38, 39.22, 38.64, 40....   
9   12          5  [53.92, 40.64, 41.97, 42.94, 40.43, 40.59, 42....   
18  24          6  [52.13, 39.32, 38.21, 43.93, 41.2, 39.2, 39.48...   
37  48          7  [49.99, 41.0, 39.34, 38.33, 39.55, 40.56, 38.4...   
75  96          8  [55.59, 43.44, 43.74, 43.19, 40.1, 40.64, 40.4...   
150 192         9  [52.38, 42.41, 41.14, 40.33, 40.99, 42.87, 42....   
301 384        10  [52.58, 41.34, 42.56, 44.27, 43.69, 43.14, 43....   
603 769        11  [52.94, 43.74, 42.29, 40.76, 41.58, 44.31, 43....   

         mean total time  
0   0             53.299  
    0             46.052  
1   1             46.453  
2   3             45.553  
4   6             41.111  
9   12            42.634  
18  24            41.917  
37  48            41.395  
75  96            43.409  
150 192           43.717  
301 384           43.510  
603 769           43.980